In [348]:
import pandas as pd
import numpy as np
import csv
from gurobipy import *
import itertools as it

# Cargo Freight Scheduling

## Part 1: Building our data

In [2]:
cargo_data = pd.read_csv('cargo-city-locations.csv')

In [3]:
cargo_data

,State,City,Latitude,Longitude
0,Alabama,Montgomery,32.377716,-86.300568
1,Alaska,Juneau,58.301598,-134.420212
2,Arizona,Phoenix,33.448143,-112.096962
3,Arkansas,Little Rock,34.746613,-92.288986
4,California,Sacramento,38.576668,-121.493629
...,...,...,...,...
63,Washington,Seattle,47.606200,-122.332100
64,DC,Washington,38.907200,-77.036900
65,Texas,El Paso,31.761900,-106.485000
66,Michigan,Detroit,42.331400,-83.045800


In [60]:
cargo_data['city_2'] = cargo_data['City']

In [59]:
# cargo_data[cargo_data['City'] == 'Annapolis']

In [58]:
# distance_matrix

In [19]:
distance_matrix = cargo_data.pivot_table(index='City', columns='city_2')['Longitude']

time_matrix = cargo_data.pivot_table(index='City', columns='city_2')['Longitude']

In [92]:
def get_distance(lat1, lon1, lat2, lon2):
    lat1 = lat1/360*2*np.pi
    lat2 = lat2/360*2*np.pi
    lon1 = lon1/360*2*np.pi
    lon2 = lon2/360*2*np.pi
    
    h = (1 - np.cos(lat2 - lat1))/2 + np.cos(lat1)*np.cos(lat2)*(1 - np.cos(lon2 - lon1))/2
    return 2*np.arcsin(np.sqrt(h))*6378.137

In [96]:
for city_ in cargo_data['City'].values:
# for city_ in cargo_data['City'].values[14:15]:
    lat2, lon2 = cargo_data[cargo_data['City'] == city_].values[0][2], \
                cargo_data[cargo_data['City'] == city_].values[0][3]
    temp_df = cargo_data.sort_values(by='City')
    temp_df['Distance'] = temp_df[['Latitude', 'Longitude']].apply(lambda x: get_distance(*x, lat2, lon2), axis=1)
#     print(city_, temp_df['Distance'].values)
    temp_df['Time'] = temp_df['Distance']/908
    distance_matrix.at[city_] = temp_df['Distance'].values
    time_matrix.at[city_] = temp_df['Time'].values
#     break

In [95]:
# temp_df.iloc[5:6, :]

In [85]:
# temp_df

In [65]:
# distance_matrix.iloc[40:41,39:42]

In [97]:
distance_matrix

city_2,Albany,Annapolis,Atlanta,Augusta,Austin,Baton Rouge,Bismarck,Boise,Boston,Carson City,...,San Francisco,San Jose,Santa Fe,Seattle,Springfield,St. Paul,Tallahassee,Topeka,Trenton,Washington
City,,,,,,,,,,,,,,,,,,,,,
Albany,0.000000,469.289632,1356.937179,370.165358,2537.150627,2059.427381,2175.329794,3411.459059,223.510737,3842.296659,...,4114.760914,4091.035823,2864.608982,3791.142308,1366.145766,1571.229834,1650.704326,1883.166681,283.617402,500.158891
Annapolis,469.289632,0.000000,915.911357,813.900245,2166.963816,1642.206499,2153.336864,3328.547576,592.664308,3703.620841,...,3967.242659,3936.220344,2620.792131,3777.405348,1135.196269,1523.532728,1187.261301,1656.507127,202.233031,47.936717
Atlanta,1356.937179,915.911357,0.000000,1720.468373,1317.172571,738.227400,2005.904553,2952.152908,1507.651971,3203.468130,...,3440.720393,3397.602797,1979.732355,3509.590804,820.621825,1453.589286,368.706560,1169.697151,1117.032750,873.725315
Augusta,370.165358,813.900245,1720.468373,0.000000,2905.432379,2428.682259,2416.385854,3669.550852,240.508453,4126.676065,...,4402.060282,4382.091151,3200.310751,4005.057037,1712.698748,1842.481938,2000.891286,2220.920446,612.800126,851.413413
Austin,2537.150627,2166.963816,1317.172571,2905.432379,0.000000,629.638304,1860.482866,2204.953939,2728.064526,2237.075287,...,2416.297401,2359.791361,973.321627,2850.514113,1289.835407,1683.849898,1292.207026,994.666588,2353.634254,2119.875710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
St. Paul,1571.229834,1523.532728,1453.589286,1842.481938,1683.849898,1622.568558,630.050985,1840.364888,1792.594808,2284.337214,...,2560.140927,2541.474938,1497.492645,2254.489125,640.070511,0.000000,1790.450539,691.112990,1588.022769,1487.473894
Tallahassee,1650.704326,1187.261301,368.706560,2000.891286,1292.207026,662.662856,2310.550808,3166.345517,1774.216974,3361.934819,...,3582.821191,3534.575901,2098.278252,3752.131520,1150.544047,1790.450539,0.000000,1413.954597,1388.137975,1151.761569
Topeka,1883.166681,1656.507127,1169.697151,2220.920446,994.666588,1040.437718,959.571082,1783.897921,2101.978433,2074.658229,...,2328.465347,2292.888005,981.443143,2344.083591,524.510024,691.112990,1413.954597,0.000000,1793.041784,1610.417419


In [98]:
time_matrix

city_2,Albany,Annapolis,Atlanta,Augusta,Austin,Baton Rouge,Bismarck,Boise,Boston,Carson City,...,San Francisco,San Jose,Santa Fe,Seattle,Springfield,St. Paul,Tallahassee,Topeka,Trenton,Washington
City,,,,,,,,,,,,,,,,,,,,,
Albany,0.000000,0.516839,1.494424,0.407671,2.794219,2.268092,2.395738,3.757114,0.246157,4.231604,...,4.531675,4.505546,3.154856,4.175267,1.504566,1.730429,1.817956,2.073972,0.312354,0.550836
Annapolis,0.516839,0.000000,1.008713,0.896366,2.386524,1.808597,2.371516,3.665801,0.652714,4.078878,...,4.369210,4.335044,2.886335,4.160138,1.250216,1.677899,1.307556,1.824347,0.222724,0.052794
Atlanta,1.494424,1.008713,0.000000,1.894789,1.450631,0.813026,2.209146,3.251270,1.660410,3.528049,...,3.789340,3.741853,2.180322,3.865188,0.903769,1.600869,0.406064,1.288213,1.230212,0.962253
Augusta,0.407671,0.896366,1.894789,0.000000,3.199815,2.674760,2.661218,4.041356,0.264877,4.544797,...,4.848084,4.826092,3.524571,4.410856,1.886232,2.029165,2.203625,2.445948,0.674890,0.937680
Austin,2.794219,2.386524,1.450631,3.199815,0.000000,0.693434,2.048990,2.428363,3.004476,2.463739,...,2.661120,2.598889,1.071940,3.139333,1.420524,1.854460,1.423135,1.095448,2.592108,2.334665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
St. Paul,1.730429,1.677899,1.600869,2.029165,1.854460,1.786970,0.693889,2.026834,1.974223,2.515790,...,2.819538,2.798981,1.649221,2.482918,0.704923,0.000000,1.971862,0.761138,1.748924,1.638187
Tallahassee,1.817956,1.307556,0.406064,2.203625,1.423135,0.729805,2.544659,3.487165,1.953983,3.702571,...,3.945838,3.892705,2.310879,4.132303,1.267119,1.971862,0.000000,1.557219,1.528786,1.268460
Topeka,2.073972,1.824347,1.288213,2.445948,1.095448,1.145857,1.056796,1.964645,2.314954,2.284866,...,2.564389,2.525207,1.080885,2.581590,0.577654,0.761138,1.557219,0.000000,1.974716,1.773587


In [99]:
time_matrix.iloc[5:6,19:21]

city_2,Des Moines,Detroit
City,,
Baton Rouge,1.385712,1.660831


In [100]:
distance_matrix.iloc[5:6,19:21]

city_2,Des Moines,Detroit
City,,
Baton Rouge,1258.226139,1508.034474


In [101]:
distance_matrix.iloc[19:21,5:6]

city_2,Baton Rouge
City,
Des Moines,1258.226139
Detroit,1508.034474


### a) Which two cities have the highest travel travel time?

In [130]:
max_time = 0.0
max_city = ''
source_city = ''
for city_ in time_matrix.index:
    time_max = time_matrix[city_].values.max()
    city_max = time_matrix[city_].index[np.argmax(time_matrix[city_].values)]
    
    if time_max > max_time:
        max_time = time_max
        max_city = city_max
        source_city = city_
    else:
        continue

In [131]:
max_time

9.068150332762624

In [132]:
max_city

'Honolulu'

In [133]:
source_city

'Augusta'

### b) Which two cities have the smallest travel time?

In [157]:
# time_matrix[city_].values

# np.where(time_matrix[city_].values, time_matrix[city_].values==0.0, 10000.0)

# a = time_matrix[city_].values

# a[np.where(a==0)]=999.0

In [152]:
min_time = 1000.0
min_city = ''
source_city = ''
for city_ in time_matrix.index:
    ax = time_matrix[city_].values.copy()
    ax[np.where(ax==0)]=999.0
    time_min = ax.min()
    city_min = time_matrix[city_].index[np.argmin(ax)]
    
    if time_min < min_time:
        min_time = time_min
        min_city = city_min
        source_city = city_
    else:
        continue

In [153]:
min_time

0.04953966019481448

In [154]:
min_city

'Trenton'

In [155]:
source_city

'Philadelphia'

### c) Which city has the smallest average travel time to all of the other cities? (Here, the average is taken over the other 67 cities.)

In [158]:
time_matrix

city_2,Albany,Annapolis,Atlanta,Augusta,Austin,Baton Rouge,Bismarck,Boise,Boston,Carson City,...,San Francisco,San Jose,Santa Fe,Seattle,Springfield,St. Paul,Tallahassee,Topeka,Trenton,Washington
City,,,,,,,,,,,,,,,,,,,,,
Albany,0.000000,0.516839,1.494424,0.407671,2.794219,2.268092,2.395738,3.757114,0.246157,4.231604,...,4.531675,4.505546,3.154856,4.175267,1.504566,1.730429,1.817956,2.073972,0.312354,0.550836
Annapolis,0.516839,0.000000,1.008713,0.896366,2.386524,1.808597,2.371516,3.665801,0.652714,4.078878,...,4.369210,4.335044,2.886335,4.160138,1.250216,1.677899,1.307556,1.824347,0.222724,0.052794
Atlanta,1.494424,1.008713,0.000000,1.894789,1.450631,0.813026,2.209146,3.251270,1.660410,3.528049,...,3.789340,3.741853,2.180322,3.865188,0.903769,1.600869,0.406064,1.288213,1.230212,0.962253
Augusta,0.407671,0.896366,1.894789,0.000000,3.199815,2.674760,2.661218,4.041356,0.264877,4.544797,...,4.848084,4.826092,3.524571,4.410856,1.886232,2.029165,2.203625,2.445948,0.674890,0.937680
Austin,2.794219,2.386524,1.450631,3.199815,0.000000,0.693434,2.048990,2.428363,3.004476,2.463739,...,2.661120,2.598889,1.071940,3.139333,1.420524,1.854460,1.423135,1.095448,2.592108,2.334665
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
St. Paul,1.730429,1.677899,1.600869,2.029165,1.854460,1.786970,0.693889,2.026834,1.974223,2.515790,...,2.819538,2.798981,1.649221,2.482918,0.704923,0.000000,1.971862,0.761138,1.748924,1.638187
Tallahassee,1.817956,1.307556,0.406064,2.203625,1.423135,0.729805,2.544659,3.487165,1.953983,3.702571,...,3.945838,3.892705,2.310879,4.132303,1.267119,1.971862,0.000000,1.557219,1.528786,1.268460
Topeka,2.073972,1.824347,1.288213,2.445948,1.095448,1.145857,1.056796,1.964645,2.314954,2.284866,...,2.564389,2.525207,1.080885,2.581590,0.577654,0.761138,1.557219,0.000000,1.974716,1.773587


In [166]:
min_mean_time = 1000.0
min_mean_city = ''
for city_ in time_matrix.index:
    ax = time_matrix[city_].values.copy()
    
    if min_mean_time > ax.mean():
        min_mean_time = ax.mean()
        min_mean_city = city_
    else:
        continue

In [167]:
min_mean_time

1.5755056594069623

In [168]:
min_mean_city

'Springfield'

## Part 2: Finding the schedule

### a) random trips

In [183]:
# np.random.seed(50)
# nCities = 68
# temp = np.random.permutation(nCities)

# order
# temp

# total_travel_time = 0.0
# for index_ in temp:
#     source_city = cargo_data['City'].values[index_]
#     try:
#         destination_city = cargo_data['City'].values[index_+1]
#     except:
#         print(index_)
#         destination_city = cargo_data['City'].values[0]
    
#     travel_time = time_matrix.loc[source_city][destination_city]
#     total_travel_time = total_travel_time + travel_time
    

# # over 1 iteration
# total_travel_time

In [320]:
np.random.seed(50)
temp_ = np.zeros((100,68))

for i in range(100):
    temp_[i] = np.random.permutation(nCities)
    
temp_ = temp_.astype(int)

In [324]:
temp_

array([[32, 29, 39, ..., 33, 11, 48],
       [58, 57, 65, ..., 47, 19, 32],
       [53, 20, 12, ..., 64, 59,  9],
       ...,
       [29, 59,  4, ...,  1, 63, 32],
       [ 9, 21, 67, ..., 31, 65, 54],
       [ 8, 54, 67, ..., 18, 59, 63]])

In [326]:
cargo_data['City'].values

array(['Montgomery', 'Juneau', 'Phoenix', 'Little Rock', 'Sacramento',
       'Denver', 'Hartford', 'Dover', 'Honolulu', 'Tallahassee',
       'Atlanta', 'Boise', 'Springfield', 'Indianapolis', 'Des Moines',
       'Topeka', 'Frankfort', 'Baton Rouge', 'Augusta', 'Annapolis',
       'Boston', 'Lansing', 'St. Paul', 'Jackson', 'Jefferson City',
       'Helena', 'Lincoln', 'Carson City', 'Concord', 'Trenton',
       'Santa Fe', 'Raleigh', 'Bismarck', 'Albany', 'Columbus',
       'Oklahoma City', 'Salem', 'Harrisburg', 'Providence', 'Columbia',
       'Pierre', 'Nashville', 'Austin', 'Salt Lake City', 'Montpelier',
       'Richmond', 'Olympia', 'Charleston', 'Madison', 'Cheyenne',
       'New York City', 'Los Angeles', 'Chicago', 'Houston',
       'Philadelphia', 'San Antonio', 'San Diego', 'Dallas', 'San Jose',
       'Jacksonville', 'Fort Worth', 'San Francisco', 'Charlotte',
       'Seattle', 'Washington', 'El Paso', 'Detroit', 'Portland'],
      dtype=object)

In [327]:
# over 100 iterations
total_travel_time = 0.0
for i in range(100):
    
#     temp = np.random.permutation(nCities)
#     print(temp)
    temp = temp_[i]
    total_travel_time_iteration = 0.0
    
    for i in range(len(temp)):
        source_city = cargo_data['City'].values[temp[i]]
        try:
            destination_city = cargo_data['City'].values[temp[i+1]]
        except:
            print(i, total_travel_time_iteration)
            destination_city = cargo_data['City'].values[temp[i]]
        
        print(source_city, destination_city)
        travel_time = time_matrix.loc[source_city][destination_city]
        total_travel_time_iteration += travel_time
    
    final_source = cargo_data['City'].values[temp[-1]]
    final_destination = cargo_data['City'].values[temp[0]]
    total_travel_time_iteration = total_travel_time_iteration + time_matrix.loc[final_source][final_destination]
    
    total_travel_time += total_travel_time_iteration
    print(total_travel_time_iteration)
    print('----')

Bismarck Trenton
Trenton Columbia
Columbia Tallahassee
Tallahassee Salem
Salem Baton Rouge
Baton Rouge Honolulu
Honolulu San Jose
San Jose Juneau
Juneau Augusta
Augusta Los Angeles
Los Angeles Jacksonville
Jacksonville Fort Worth
Fort Worth New York City
New York City Jackson
Jackson Cheyenne
Cheyenne Houston
Houston Austin
Austin Indianapolis
Indianapolis Columbus
Columbus Helena
Helena San Antonio
San Antonio Detroit
Detroit Jefferson City
Jefferson City Montpelier
Montpelier Providence
Providence Chicago
Chicago Frankfort
Frankfort Atlanta
Atlanta El Paso
El Paso Little Rock
Little Rock Pierre
Pierre Springfield
Springfield Charlotte
Charlotte Philadelphia
Philadelphia Santa Fe
Santa Fe Topeka
Topeka Dallas
Dallas Annapolis
Annapolis Olympia
Olympia Montgomery
Montgomery Nashville
Nashville Lansing
Lansing San Diego
San Diego Salt Lake City
Salt Lake City Boston
Boston San Francisco
San Francisco Portland
Portland Lincoln
Lincoln Carson City
Carson City Concord
Concord Oklahoma City

Sacramento Salt Lake City
Salt Lake City Albany
Albany Harrisburg
Harrisburg Portland
Portland Annapolis
Annapolis Charlotte
Charlotte Trenton
Trenton Pierre
Pierre Boise
Boise Augusta
Augusta Dover
Dover Frankfort
Frankfort Helena
Helena San Jose
San Jose Carson City
Carson City Atlanta
Atlanta Springfield
Springfield Detroit
Detroit Fort Worth
Fort Worth Olympia
Olympia San Antonio
San Antonio Des Moines
Des Moines Philadelphia
Philadelphia Boston
Boston Raleigh
Raleigh Santa Fe
Santa Fe St. Paul
St. Paul Denver
Denver Washington
Washington Jefferson City
Jefferson City Montgomery
67 152.04893336511617
Montgomery Montgomery
155.8466178203189
----
Topeka Houston
Houston Sacramento
Sacramento Montgomery
Montgomery Madison
Madison Cheyenne
Cheyenne Columbus
Columbus Philadelphia
Philadelphia El Paso
El Paso Salt Lake City
Salt Lake City Jackson
Jackson Pierre
Pierre Frankfort
Frankfort Los Angeles
Los Angeles Detroit
Detroit St. Paul
St. Paul Charlotte
Charlotte Bismarck
Bismarck Chicag

Philadelphia Oklahoma City
Oklahoma City San Francisco
San Francisco Tallahassee
Tallahassee Chicago
Chicago Montpelier
Montpelier Washington
Washington Richmond
Richmond Columbia
Columbia New York City
New York City Olympia
Olympia Houston
67 151.297307334991
Houston Houston
153.89916639927193
----
Seattle Phoenix
Phoenix Atlanta
Atlanta Oklahoma City
Oklahoma City Detroit
Detroit Charleston
Charleston Richmond
Richmond Trenton
Trenton Austin
Austin Little Rock
Little Rock Hartford
Hartford Raleigh
Raleigh Boise
Boise Houston
Houston Santa Fe
Santa Fe Pierre
Pierre Topeka
Topeka Lincoln
Lincoln Baton Rouge
Baton Rouge Washington
Washington Salt Lake City
Salt Lake City Los Angeles
Los Angeles Denver
Denver El Paso
El Paso Des Moines
Des Moines Annapolis
Annapolis Bismarck
Bismarck Madison
Madison San Diego
San Diego Springfield
Springfield New York City
New York City Sacramento
Sacramento Harrisburg
Harrisburg Montpelier
Montpelier Frankfort
Frankfort Dallas
Dallas Charlotte
Charlotte

Salt Lake City Richmond
Richmond Augusta
Augusta Austin
Austin Lansing
Lansing Carson City
Carson City Lincoln
Lincoln Los Angeles
Los Angeles Indianapolis
Indianapolis Baton Rouge
Baton Rouge Sacramento
Sacramento Montpelier
Montpelier Des Moines
Des Moines Montgomery
Montgomery Dover
Dover San Diego
San Diego Chicago
Chicago Salem
Salem Trenton
Trenton Charlotte
Charlotte Columbia
Columbia Raleigh
Raleigh Nashville
Nashville Atlanta
Atlanta Jackson
Jackson Honolulu
Honolulu Topeka
Topeka Springfield
Springfield Pierre
Pierre New York City
New York City Boston
Boston Jacksonville
67 155.42472365748955
Jacksonville Jacksonville
156.90213966331063
----
Honolulu Denver
Denver Columbus
Columbus Portland
Portland Hartford
Hartford Frankfort
Frankfort New York City
New York City Trenton
Trenton Harrisburg
Harrisburg Annapolis
Annapolis Des Moines
Des Moines Sacramento
Sacramento Bismarck
Bismarck Concord
Concord Detroit
Detroit Baton Rouge
Baton Rouge Seattle
Seattle El Paso
El Paso Little 

Harrisburg Charleston
Charleston Frankfort
Frankfort Jacksonville
Jacksonville Salt Lake City
Salt Lake City Austin
Austin Nashville
Nashville Lansing
Lansing Baton Rouge
Baton Rouge San Antonio
San Antonio Fort Worth
Fort Worth Dover
Dover Sacramento
67 153.86512405869726
Sacramento Sacramento
155.43872551389333
----
Cheyenne Lincoln
Lincoln Oklahoma City
Oklahoma City Raleigh
Raleigh Concord
Concord Seattle
Seattle Austin
Austin Salt Lake City
Salt Lake City Hartford
Hartford Washington
Washington Bismarck
Bismarck Nashville
Nashville Chicago
Chicago Madison
Madison San Diego
San Diego Philadelphia
Philadelphia Detroit
Detroit Charleston
Charleston Tallahassee
Tallahassee Topeka
Topeka Charlotte
Charlotte Montgomery
Montgomery Phoenix
Phoenix San Jose
San Jose Springfield
Springfield Frankfort
Frankfort Harrisburg
Harrisburg Baton Rouge
Baton Rouge Providence
Providence Richmond
Richmond Jackson
Jackson Los Angeles
Los Angeles Salem
Salem Albany
Albany Trenton
Trenton Indianapolis
In

Charleston Bismarck
Bismarck Jefferson City
Jefferson City Oklahoma City
Oklahoma City Des Moines
Des Moines San Antonio
San Antonio Atlanta
Atlanta Albany
Albany Denver
Denver El Paso
El Paso Austin
Austin Richmond
Richmond San Jose
San Jose Pierre
Pierre Chicago
Chicago Juneau
Juneau Raleigh
Raleigh Augusta
Augusta Salt Lake City
Salt Lake City Jackson
Jackson Topeka
Topeka Phoenix
Phoenix Frankfort
Frankfort Fort Worth
Fort Worth Carson City
Carson City Columbia
Columbia Portland
Portland Montgomery
Montgomery St. Paul
St. Paul Helena
Helena Boise
Boise Lansing
Lansing Concord
Concord Indianapolis
Indianapolis Springfield
Springfield Charlotte
Charlotte Washington
Washington Providence
Providence Baton Rouge
Baton Rouge Cheyenne
Cheyenne Boston
Boston New York City
New York City Jacksonville
Jacksonville Hartford
Hartford San Francisco
San Francisco Dover
Dover Harrisburg
Harrisburg Tallahassee
Tallahassee Santa Fe
Santa Fe Olympia
Olympia Sacramento
Sacramento Dallas
Dallas Annapol

Helena Olympia
Olympia San Jose
San Jose San Diego
San Diego El Paso
El Paso Harrisburg
Harrisburg Boston
Boston Little Rock
Little Rock Annapolis
Annapolis Fort Worth
Fort Worth San Antonio
San Antonio Oklahoma City
Oklahoma City Dallas
Dallas Baton Rouge
Baton Rouge Carson City
Carson City Salt Lake City
Salt Lake City Charlotte
Charlotte New York City
New York City Augusta
Augusta Houston
Houston Hartford
Hartford Topeka
Topeka Santa Fe
Santa Fe Nashville
Nashville Phoenix
Phoenix Jefferson City
Jefferson City Philadelphia
Philadelphia Des Moines
Des Moines Los Angeles
Los Angeles Charleston
Charleston Albany
Albany Columbia
Columbia Lincoln
Lincoln Detroit
Detroit Tallahassee
Tallahassee Montpelier
Montpelier Austin
Austin Portland
Portland Columbus
Columbus San Francisco
San Francisco Denver
Denver Salem
Salem Madison
67 128.1832751494454
Madison Madison
129.52809663226256
----
Boise Los Angeles
Los Angeles Albany
Albany Charlotte
Charlotte Columbus
Columbus El Paso
El Paso Charle

San Diego Helena
Helena Portland
Portland Dover
Dover Raleigh
Raleigh San Antonio
San Antonio Cheyenne
Cheyenne Carson City
Carson City Philadelphia
Philadelphia Juneau
Juneau Concord
Concord Hartford
Hartford Nashville
Nashville Salem
Salem Los Angeles
Los Angeles Albany
Albany Des Moines
Des Moines Boston
Boston Jacksonville
Jacksonville Atlanta
Atlanta St. Paul
St. Paul Columbia
Columbia San Jose
67 140.48220333352904
San Jose San Jose
141.74168458293485
----
Trenton Jacksonville
Jacksonville Sacramento
Sacramento Raleigh
Raleigh Montgomery
Montgomery Hartford
Hartford Philadelphia
Philadelphia Fort Worth
Fort Worth Dover
Dover Helena
Helena Carson City
Carson City Baton Rouge
Baton Rouge Los Angeles
Los Angeles Albany
Albany Providence
Providence Columbia
Columbia San Jose
San Jose Honolulu
Honolulu Denver
Denver Tallahassee
Tallahassee Boise
Boise El Paso
El Paso Lincoln
Lincoln New York City
New York City St. Paul
St. Paul Topeka
Topeka San Antonio
San Antonio Houston
Houston Chi

In [330]:
round(total_travel_time/100, 1)

149.5

### b) nearest trips

In [268]:
start_list = ['Los Angeles']
city_to_travel = []
travel_time = 0.0

while len(start_list) < 68:
    city_ = start_list[-1]
#     ax = time_matrix[city_].values.copy()
#     ax[np.where(ax==0.0)]=999.0
#     time_min = ax.min()
#     city_min = time_matrix[city_].index[np.argmin(ax)]
#     print(city_)
    remaining_cities = list(set(time_matrix.columns) - set(start_list))
    ax = time_matrix[city_].loc[remaining_cities].sort_values()
#     print(ax)
    time_min = ax[0]
    city_min = ax.index[0]
    
    start_list.append(city_min)
    travel_time = travel_time + time_min
    city_to_travel = remaining_cities
#     print(city_min, time_min)
#     break

In [285]:
travel_time = travel_time + time_matrix[start_list[-1]][start_list[0]]
print('Total time = ', travel_time)

Total time =  35.851706754982374


In [289]:
print('Order to travel:',start_list+[start_list[0]])

Order to travel: ['Los Angeles', 'San Diego', 'Phoenix', 'El Paso', 'Santa Fe', 'Denver', 'Cheyenne', 'Pierre', 'Bismarck', 'St. Paul', 'Madison', 'Chicago', 'Indianapolis', 'Frankfort', 'Columbus', 'Charleston', 'Charlotte', 'Columbia', 'Raleigh', 'Richmond', 'Washington', 'Annapolis', 'Dover', 'Philadelphia', 'Trenton', 'New York City', 'Hartford', 'Providence', 'Boston', 'Concord', 'Montpelier', 'Albany', 'Augusta', 'Harrisburg', 'Detroit', 'Lansing', 'Springfield', 'Jefferson City', 'Topeka', 'Lincoln', 'Des Moines', 'Oklahoma City', 'Fort Worth', 'Dallas', 'Austin', 'San Antonio', 'Houston', 'Baton Rouge', 'Jackson', 'Little Rock', 'Nashville', 'Atlanta', 'Montgomery', 'Tallahassee', 'Jacksonville', 'Salt Lake City', 'Boise', 'Helena', 'Seattle', 'Olympia', 'Portland', 'Salem', 'Carson City', 'Sacramento', 'San Francisco', 'San Jose', 'Juneau', 'Honolulu', 'Los Angeles']


### c) optimization problem

In [338]:
city_pairs = list(it.permutations(np.arange(68), 2))

In [353]:
def eliminateSubtours(model, where):
    if (where == GRB.Callback.MIPSOL):
        x_val = model.cbGetSolution(x)
        sequence = [ (i,j) for (i,j) in city_pairs if x_val[i,j] > 0.5]
        subtour_list = getSubtours(sequence)
        if (len(subtour_list) > 1):
            for subtour in subtour_list:
                model.cbLazy( sum(x[i,j] for i in subtour for j in subtour if i != j) <= len(subtour) - 1)

In [356]:
def getSubtours(sequence):
    subtour_list = []
    unvisited = list(range(nCities))
    
    while ( len(unvisited) > 0 ):
        node = unvisited.pop()
        
        subtour = []
        subtour.append(node)
        
        next_node = list(filter(lambda t: t[0] == node, sequence))[0][1]
        
        while (next_node in unvisited):
            subtour.append(next_node)
            unvisited.remove(next_node)
            next_node = list(filter(lambda t: t[0] == next_node, sequence))[0][1]
            
        subtour_list.append(subtour)
    
    return subtour_list

In [357]:
m = Model()

x = m.addVars(city_pairs, vtype = GRB.BINARY)

for i in range(nCities):
    m.addConstr( sum(x[i,j] for j in range(nCities) if j != i ) == 1)
    m.addConstr( sum(x[j,i] for j in range(nCities) if j != i ) == 1)

# m.setObjective(sum(travel_time_dict[i,j] * x[i,j] for (i,j) in od_pairs), GRB.MINIMIZE)
m.setObjective(sum(time_matrix.iloc[i,j] * x[i,j] for (i,j) in city_pairs), GRB.MINIMIZE)

m.update()

# Enable lazy constraints
m.params.LazyConstraints = 1

# Supply the callback to Gurobi:
m.optimize(eliminateSubtours)

Set parameter LazyConstraints to value 1
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 136 rows, 4556 columns and 9112 nonzeros
Model fingerprint: 0xd6bd90c1
Variable types: 0 continuous, 4556 integer (4556 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [5e-02, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.01s
Presolved: 136 rows, 4556 columns, 9112 nonzeros
Variable types: 0 continuous, 4556 integer (4556 binary)

Root relaxation: objective 2.729102e+01, 170 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   29.74856    0   70          -   29.74856      -     -    0s
     0     0   29.97845    0   26          -   29.97845      -   

In [359]:
sequence = [ (i,j) for (i,j) in city_pairs if x[i,j].x > 0.5]
subtour_list = getSubtours(sequence)
complete_tour = subtour_list[0]
complete_tour.append( complete_tour[0] )
print("Complete tour: ", complete_tour)

Complete tour:  [67, 52, 51, 11, 14, 32, 64, 40, 2, 42, 24, 30, 62, 19, 36, 65, 33, 37, 31, 5, 29, 56, 4, 23, 17, 44, 60, 22, 47, 57, 38, 9, 53, 59, 58, 28, 34, 61, 45, 49, 54, 7, 27, 55, 18, 12, 48, 6, 63, 39, 13, 35, 20, 15, 10, 25, 0, 41, 3, 16, 8, 50, 26, 43, 66, 46, 21, 1, 67]


In [366]:
print('Travel time of optimized sequence: ', round(m.objVal, 2))

Travel time of optimized sequence:  30.55


In [363]:
time_matrix.columns[complete_tour]

Index(['Washington', 'Richmond', 'Raleigh', 'Charlotte', 'Columbia',
       'Jacksonville', 'Tallahassee', 'Montgomery', 'Atlanta', 'Nashville',
       'Frankfort', 'Indianapolis', 'Springfield', 'Des Moines', 'Lincoln',
       'Topeka', 'Jefferson City', 'Little Rock', 'Jackson', 'Baton Rouge',
       'Houston', 'San Antonio', 'Austin', 'Fort Worth', 'Dallas',
       'Oklahoma City', 'Santa Fe', 'El Paso', 'Phoenix', 'San Diego',
       'Los Angeles', 'Carson City', 'Sacramento', 'San Jose', 'San Francisco',
       'Honolulu', 'Juneau', 'Seattle', 'Olympia', 'Portland', 'Salem',
       'Boise', 'Helena', 'Salt Lake City', 'Denver', 'Cheyenne', 'Pierre',
       'Bismarck', 'St. Paul', 'Madison', 'Chicago', 'Lansing', 'Detroit',
       'Columbus', 'Charleston', 'Harrisburg', 'Albany', 'Montpelier',
       'Augusta', 'Concord', 'Boston', 'Providence', 'Hartford',
       'New York City', 'Trenton', 'Philadelphia', 'Dover', 'Annapolis',
       'Washington'],
      dtype='object', name='cit

# 2 Winning the NBA championship

In [367]:
nba_stats = pd.read_csv('nba-stats-2013-2019.csv')
players_with_pos = pd.read_csv('nba-players-2018-2019-with-pos.csv')

In [398]:
players_to_keep = 15
df = players_with_pos[['X3PA', 'X2PA', 'FTA', 'ORB', 'DRB', 'AST', 'STL', 'BLK', 'TOV', 'PF']]

coeffs = np.array([0.69129,0.36184,0.78122,-0.20277,0.61064,0.91510,0.50506,0.07975,-0.62701,0.30420])

intercept = np.ones(df.shape[0]).reshape(df.shape[0], 1)

In [406]:
players_with_pos['Pos'].value_counts()

SG    134
PG    107
PF    104
C      94
SF     83
Name: Pos, dtype: int64

In [409]:
players_with_pos['position_SG'] = np.where(players_with_pos['Pos'] == 'SG', 1, 0)
players_with_pos['position_PG'] = np.where(players_with_pos['Pos'] == 'PG', 1, 0)
players_with_pos['position_PF'] = np.where(players_with_pos['Pos'] == 'PF', 1, 0)
players_with_pos['position_C'] = np.where(players_with_pos['Pos'] == 'C', 1, 0)
players_with_pos['position_SF'] = np.where(players_with_pos['Pos'] == 'SF', 1, 0)

In [417]:
m = Model()

#Setting output flag to 0
m.Params.OutputFlag = 1

# Create the decision variables
x = m.addVars(players_with_pos.shape[0], vtype = GRB.BINARY)

# Create the constraints:

# Total Player Constraint
m.addConstr(sum(x[i] for i in range(players_with_pos.shape[0])) == 15)

# Salary Constraint
# m.addConstr(sum(players_with_pos.values[i,-6] * x[i] for i in range(players_with_pos.shape[0])) <= 100000000)
m.addConstr(sum(players_with_pos['Salary'].values[i] * x[i] for i in range(players_with_pos.shape[0])) <= 100000000)

# Position Constraints
m.addConstr(sum(x[i]*players_with_pos['position_SG'].values[i] for i in range(players_with_pos.shape[0])) == 3)
m.addConstr(sum(x[i]*players_with_pos['position_PG'].values[i] for i in range(players_with_pos.shape[0])) == 3)
m.addConstr(sum(x[i]*players_with_pos['position_PF'].values[i] for i in range(players_with_pos.shape[0])) == 3)
m.addConstr(sum(x[i]*players_with_pos['position_C'].values[i] for i in range(players_with_pos.shape[0])) == 3)
m.addConstr(sum(x[i]*players_with_pos['position_SF'].values[i] for i in range(players_with_pos.shape[0])) == 3)


m.setObjective(513.82999 + sum(x[i] * np.dot(df.values[i], coeffs) for i in range(players_with_pos.shape[0])), GRB.MAXIMIZE)

# Update and solve
m.update()

print("Update completed")
m.optimize()

print('Optimization done!')

Update completed
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 522 columns and 1566 nonzeros
Model fingerprint: 0xdc2ded83
Variable types: 0 continuous, 522 integer (522 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+07]
  Objective range  [2e-01, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 1e+08]
Found heuristic solution: objective 6837.4563700
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 6 rows, 522 columns, 1044 nonzeros
Variable types: 0 continuous, 522 integer (522 binary)
Found heuristic solution: objective 18867.729990

Root relaxation: objective 2.069517e+04, 18 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 20695.1697    0    2 18867

In [418]:
dream_team = [i for i in range(players_with_pos.shape[0]) if x[i].x > 0.5]
dream_team

[17, 59, 138, 167, 286, 347, 384, 427, 439, 457, 471, 483, 487, 511, 517]

In [424]:
players_with_pos.iloc[dream_team][['Player', 'Salary', 'Pos']].sort_values(by='Salary', ascending=False)

,Player,Salary,Pos
17,Giannis Antetokounmpo,24157304.0,PF
483,Nikola Vucevic,12750000.0,C
487,Kemba Walker,12000000.0,PG
471,Karl-Anthony Towns,7839435.0,C
457,Jayson Tatum,6700800.0,SF
138,Luka Doncic,6560640.0,SG
167,De'Aaron Fox,5470920.0,PG
517,Trae Young,5356440.0,PG
511,Justise Winslow,3448926.0,SF
59,Devin Booker,3314365.0,SG


In [421]:
print('The predicted # of points team will score:' , round(m.objval))

The predicted # of points team will score: 20578
